In [2]:
import numpy as np
from numpy.lib.stride_tricks import as_strided

array = np.arange(16).reshape((4,4))
print(f"strides={array.strides}")
array

strides=(32, 8)


array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [12, 13, 14, 15]])

### Calculamos strides manualmente 

In [55]:
itemsize = array.itemsize
window   = as_strided(
    array,shape=(2,2,2,2),
    strides=(itemsize*8,itemsize*2,itemsize*4,itemsize*1)
)

print(f"strides={window.strides}")
window

strides=(64, 16, 32, 8)


array([[[[ 0,  1],
         [ 4,  5]],

        [[ 2,  3],
         [ 6,  7]]],


       [[[ 8,  9],
         [12, 13]],

        [[10, 11],
         [14, 15]]]])

### Obtenemos forma de batch

In [63]:
window_batch = as_strided(window,shape=(4,2,2),strides=(32,16,8))
print(f"strides={window_batch.strides}")
window_batch

strides=(32, 16, 8)


array([[[ 0,  1],
        [ 2,  3]],

       [[ 4,  5],
        [ 6,  7]],

       [[ 8,  9],
        [10, 11]],

       [[12, 13],
        [14, 15]]])

### Regresamos a original (ventana)

In [48]:
window_restaurado = as_strided(
    window_batch,shape=(2,2,2,2),
    strides=(itemsize*8,itemsize*2,itemsize*4,itemsize*1)
)
print(f"strides={window_restaurado.strides}")
window_restaurado

strides=(64, 16, 32, 8)


array([[[[ 0,  1],
         [ 4,  5]],

        [[ 2,  3],
         [ 6,  7]]],


       [[[ 8,  9],
         [12, 13]],

        [[10, 11],
         [14, 15]]]])

### Restauramos al array original

In [57]:
as_strided(window_restaurado,shape=array.shape,strides=array.strides)

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [12, 13, 14, 15]])

In [57]:
def num_ventanas_axis(shape_axis,ventana,solapamiento):
    a = shape_axis - 2*solapamiento
    b = ventana - 2*solapamiento
    if a%b != 0:
        raise ValueError(f"El tamaño de los axis debe de ser multiplo con {b}")
    return a//b

def stride(array,ventana=3,solapamiento=0):
    shape   = array.shape
    itemsize = array.itemsize
    # Calculamos número de ventanas
    shape_out0 = num_ventanas_axis(shape[0],ventana,solapamiento)
    shape_out1 = num_ventanas_axis(shape[1],ventana,solapamiento)
    shape_out  = (shape_out0,shape_out1,ventana,ventana)
    # Calculamos los strides.
    stride0 = itemsize*(ventana-2*solapamiento)*shape[1]
    stride1 = itemsize*(ventana-2*solapamiento)
    stride2 = itemsize*shape[1]
    stride3 = itemsize*1
    strides_out = (stride0,stride1,stride2,stride3)
    return as_strided(array,shape_out,strides_out),shape


def reshape2batch(array):
    """
    Input window (x_array,y_array,ventana,ventana)
    
    Returna:
        * array en forma de batch
        * shape del array en forma ventana
        * strides del array en forma ventana
    """
    shape = array.shape
    num_batches = shape[0]*shape[1]
    ventana = shape[2]
    array_batch = array.reshape(num_batches,ventana,ventana)
    return array_batch,shape

def reshape2window(array,shape):
    """
    Input batch (num_batch,ventana,ventana)
    """
    return array.reshape(shape)

    
def reverse_stride(array,solapamiento=0):
    """
    Input:
        * shape window
    """
    # Caracterizamos el array de entrada
    shape = array.shape
    x_windows = shape[0]
    y_windows = shape[1]
    ventana   = shape[2] - solapamiento*2 
    
    # Creamos el lienzo en blanco para pintar
    array_output = np.zeros((ventana*x_windows,ventana*y_windows))
    
    # Pintamos
    for i in range(x_windows):
        for j in range(y_windows):
            if solapamiento == 0:
                array_output[i*ventana:(i+1)*ventana,j*ventana:(j+1)*ventana] = array[i,j,:,:]
            else:
                array_output[i*ventana:(i+1)*ventana,j*ventana:(j+1)*ventana] = array[i,j,solapamiento:-solapamiento,solapamiento:-solapamiento]
    return array_output

In [45]:
array = np.arange(108).reshape((9,12))
print(array)
window , shape = stride(array,ventana=3,solapamiento=1)
print("--------------")
print(window)


[[  0   1   2   3   4   5   6   7   8   9  10  11]
 [ 12  13  14  15  16  17  18  19  20  21  22  23]
 [ 24  25  26  27  28  29  30  31  32  33  34  35]
 [ 36  37  38  39  40  41  42  43  44  45  46  47]
 [ 48  49  50  51  52  53  54  55  56  57  58  59]
 [ 60  61  62  63  64  65  66  67  68  69  70  71]
 [ 72  73  74  75  76  77  78  79  80  81  82  83]
 [ 84  85  86  87  88  89  90  91  92  93  94  95]
 [ 96  97  98  99 100 101 102 103 104 105 106 107]]
--------------
[[[[  0   1   2]
   [ 12  13  14]
   [ 24  25  26]]

  [[  1   2   3]
   [ 13  14  15]
   [ 25  26  27]]

  [[  2   3   4]
   [ 14  15  16]
   [ 26  27  28]]

  [[  3   4   5]
   [ 15  16  17]
   [ 27  28  29]]

  [[  4   5   6]
   [ 16  17  18]
   [ 28  29  30]]

  [[  5   6   7]
   [ 17  18  19]
   [ 29  30  31]]

  [[  6   7   8]
   [ 18  19  20]
   [ 30  31  32]]

  [[  7   8   9]
   [ 19  20  21]
   [ 31  32  33]]

  [[  8   9  10]
   [ 20  21  22]
   [ 32  33  34]]

  [[  9  10  11]
   [ 21  22  23]
   [ 33  34  3

In [61]:
array = np.arange(108).reshape((9,12))
array = np.copy(array[:,:9])
print(array)
# Obtenemos ventana
window , shape  = stride(array,ventana=3,solapamiento=1)
# Obtenemos forma batch
window_batch , shape_window = reshape2batch(window)
#window_batch = window.reshape((4,2,2))
# Revertimos a forma ventana
window = reshape2window(window_batch,shape_window)
#window = window_batch.reshape((2,2,2,2))
# Revertimos array
rev_array = reverse_stride(window,solapamiento=1)

print("--------------")
print(rev_array)

[[  0   1   2   3   4   5   6   7   8]
 [ 12  13  14  15  16  17  18  19  20]
 [ 24  25  26  27  28  29  30  31  32]
 [ 36  37  38  39  40  41  42  43  44]
 [ 48  49  50  51  52  53  54  55  56]
 [ 60  61  62  63  64  65  66  67  68]
 [ 72  73  74  75  76  77  78  79  80]
 [ 84  85  86  87  88  89  90  91  92]
 [ 96  97  98  99 100 101 102 103 104]]
--------------
[[13. 14. 15. 16. 17. 18. 19.]
 [25. 26. 27. 28. 29. 30. 31.]
 [37. 38. 39. 40. 41. 42. 43.]
 [49. 50. 51. 52. 53. 54. 55.]
 [61. 62. 63. 64. 65. 66. 67.]
 [73. 74. 75. 76. 77. 78. 79.]
 [85. 86. 87. 88. 89. 90. 91.]]


In [56]:
a = np.arange(10)
    
a[0:]

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])